# Explore USGS Terrain COGs
This 1 arc second (~10m) resolution terrain data as 32-bit cloud-optimized geotiff in a public AWS bucket.  Each file is a 1x1 degree square. 

In [ ]:
import hvplot.xarray
import fsspec
import os
import rioxarray

#### Use the Planetary Computer STAC catalog for searching
(until we get a STAC catalog for AWS)

In [ ]:
import pystac_client
import stackstac

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/"
)

In [ ]:
threedep = catalog.get_child(id="3dep-seamless")

In [ ]:
threedep.extent.spatial.to_dict()

In [ ]:
latitude = 43.79
longitude = -110.68
buffer = 0.5
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())
print(f"{len(items)} items found")

In [ ]:
items_low_res = [
    item.to_dict()
    for item in items
    if item.properties["gsd"] == 30
]
items_high_res = [
    item.to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

In [ ]:
import stac_vrt

%time vrt = stac_vrt.build_vrt(items_low_res, data_type="Byte")

In [ ]:
low_res_data = stackstac.stack(items_low_res[0]).squeeze()

In [ ]:
low_res_data

In [ ]:
point = {"type": "Point", "coordinates": [-110.6818, 43.7904]}

search = catalog.search(collections=["3dep-seamless"], intersects=point, limit=1)
dem_item = next(search.get_items())

dem_item

In [ ]:
threedep.extent.spatial.to_dict()

In [ ]:
azure_url = dem_item.assets['data'].href
azure_url

In [ ]:
azure_prefix='https://ai4edataeuwest.blob.core.windows.net/3dep/Elevation/'
aws_prefix='s3://prd-tnm/StagedProducts/Elevation/'

In [ ]:
aws_url = azure_url.replace(azure_prefix,aws_prefix)
aws_url

Use GDAL settings from [COG Best Practices](https://github.com/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb)

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE

In [ ]:
da = rioxarray.open_rasterio(aws_url, masked=True).squeeze(dim='band', drop=True)

In [ ]:
da

Mask out the ocean

In [ ]:
da = da.where(da>0)

Visualize with hvplot from the holoviz.org tool suite

In [ ]:
da.hvplot.image(x='x',y='y', geo=True, rasterize=True, cmap='turbo', tiles='OSM', 
                alpha=0.7, frame_width=700)